# 🌊 Deep Dive Into Isolation Forest
> **작성자 : 오수지 (2022020660)**

안녕하세요, 고려대학교 산업경영공학부 DSBA 연구실 석사과정 오수지입니다.  
이번 노트북 튜토리얼에선 Model-based Anomaly Detection 방법론, 그 중에서도 **`Isolation Forest`**에 대해 코드를 통해 알아볼 예정입니다. (🔎 Anomaly Detection에 대한 기본적인 내용은 [유튜브 튜토리얼 영상](https://youtu.be/XshinhpMrLQ)을 참고해주세요.)

Isolation Forest는 매우 직관적이고, 이상치 탐지 대회에서 아직까지도 많이 사용되고 있는 방법론인데요! 여러 개의 Decision Tree를 지속적으로 분기시키면서 모든 데이터 관측치의 고립 정도 여부에 따라 이상치를 판별하는 방법입니다. 쉽게 생각해보자면 이상치 데이터일수록 다른 정상 데이터와 다른 특성을 띌테니 빨리 고립될 것이고, 정상 데이터일수록 서로서로 비슷비슷할테니 늦게 고립되겠죠?! 즉, **특정 데이터가 고립되는 leaf node까지의 거리**를 `Anomaly Score`로 정의하고, 일찍 고립될수록, 즉 root node로부터 leaf node까지의 평균 거리가 짧을수록 anomaly score가 높아지게 됩니다.

근데 수업을 들으면서도, 유튜브 동영상을 만들면서도 전 한 가지 의문점이 들었습니다. 분기의 기준이 되는 Attribute를 랜덤하게 선택하지 않으면 어떨까요?  Isolation Forest는 Decision Tree를 기반으로 하는 방법론이고, Decision Tree의 분기 기준은 Information gain, Gini Impurity, Chi-Square 등 여러가지가 있는만큼 만약 랜덤하게 하지 않으면 결과가 달라지지 않을까요?

그러기 위해서 일단 (1) Isolation Forest를 Scratch부터 구현한 레포를 pseudo code와 대조해가며 line by line으로 이해를 하고, (2) Information Gain을 기준으로 분기하는 코드를 추가한 다음 (3) 두 방식간 성능 차이를 확인해보겠습니다 😀

<img src="images/if.png" width="550">

**📌 이번 튜토리얼의 목표**
>1. Isolation Forest from Scratch
>2. Isolation Forest without Randomness from Scratch
>3. 1, 2번 간 성능 비교

**📌 Reference**
- https://github.com/mgckind/iso_forest
- https://github.com/sahandha/eif
- https://hongl.tistory.com/150

## 🛠 환경 설정

In [1]:
import numpy as np
import pandas as pd
import random
import easydict
import wandb
import nltk
import math
from tqdm.auto import tqdm
import os
import pickle
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
warnings.filterwarnings(action='ignore') 
import copy
import seaborn as sb
import matplotlib.pyplot as plt
sb.set_style(style="whitegrid")
sb.set_color_codes()

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

In [2]:
def seed_everything(seed: int = 1201):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
seed_everything()

## 1️⃣ 데이터 준비
> Dacon에서 열린 **`월간 데이콘 신용카드 사기 거래 탐지 AI 경진대회`**의 신용카드 거래 데이터를 사용하겠습니다! (대회에서 제공한 데이터의 원 출처는 대회 데이터 설명 탭에 나와있습니다.) 

**1. 학습(Train) 데이터셋 (113842개)**  

파일명: train.csv  
설명: 정상, 사기 거래의 여부를 알 수 없는(대부분 정상 거래) 신용 카드 데이터 (Unlabeled)  
ID : 신용 카드 거래 ID  
Column ('V1', 'V2', 'V3', ... ,'V30) : 비식별화된 신용 카드 거래 Feature  


**2. 검증(Validation) 데이터셋 (28462개)**   

파일명: val.csv  
설명: 정상, 사기 거래의 여부가 포함된 신용 카드 데이터
ID : 신용 카드 거래 ID   
Column ('V1', 'V2', 'V3', ... ,'V30) : 비식별화된 신용 카드 거래 Feature  
Class : 신용 카드 거래의 정상, 사기 여부 (정상 : 0, 사기 : 1)  

In [3]:
train_dataset = pd.read_csv("./data/credit_card_fraud_detection/train.csv")
val_dataset = pd.read_csv("./data/credit_card_fraud_detection/val.csv")

In [4]:
train_dataset = train_dataset.drop(columns=['ID'])
val_dataset_label = val_dataset['Class']
val_dataset_attributes = val_dataset.drop(columns=['Class', 'ID'])

## 1️⃣ Isolation Forest from Scratch
> 코드는 https://github.com/mgckind/iso_forest 에서 가져왔습니다. 다만, 코드에 주석이 없어 알고리즘을 알고 코드를 봐도 이해가 어렵습니다. 그래서 본 목차는 위의 코드를 기반으로 신용카드 거래 데이터셋에 맞는 추가적인 리팩토링 및 pseudo code와 함께 line by line으로 함께 이해하는 것이 목표입니다.

### 🔎 Node
> Tree의 각 Node에는 어떤 정보가 포함될까요?
- 여기서 `External Node`란 자식 노드가 없는 노드(즉, Leaf Node), `Internal Node`란 적어도 하나의 자식 노드가 있는 노드를 뜻합니다.

In [5]:
class Node(object):
    def __init__(self, X, q, p, e, left, right, node_type = ''):
        self.e = e  # 현재 tree height
        self.size = len(X) # 데이터에 포함된 object 개수
        self.q = q  # split attribute
        self.p = p  # split point
        self.left = left  # 현재 Node의 Left Node
        self.right = right  # 현재 Node의 Right Node
        self.ntype = node_type  # 현재 Node의 Type (External Node or Internal Node)

### 🔎 iTree
<img src="images/if_itree.png" width="450">

In [6]:
class iTree(object):
    def __init__(self, X, e, l):
        self.e = e  # 현재 tree height
        self.X = X  # 데이터
        self.size = len(X) # 데이터에 포함된 object 개수
        self.Q = list(X.columns)  # 데이터의 attributes
        self.l = l  # 종료 조건에 사용될 tree의 height limit
        self.p = None  # split point (max(values of q), min(values of q) 중 랜덤하게 선택됨)
        self.q = None  # split attribute (Q에서 랜덤하게 선택됨)
        self.exnodes = 0  # external node
        self.root = self.make_tree(X, e, l)
        

    def make_tree(self, X, e, l, split_criteria='random'):
        self.e = e
        
        # 종료 조건
        # 1. 현재 tree height이 사전 지정한 height limit에 도달한 경우
        # 2. leaf node에 데이터가 한개만 남겨진 경우
        if e >= l or len(X) <= 1:
            left = None
            right = None
            self.exnodes += 1
            return Node(X, self.q, self.p, e, left, right, node_type = 'exNode')
        else:
            # Split하는 기준이 되는 Attribute를 데이터의 Attributes 중에서 랜덤하게 선택
            self.q = random.choice(self.Q)
            mini = X[:][self.q].min()
            maxi = X[:][self.q].max()
            if mini==maxi:
                left = None
                right = None
                self.exnodes += 1
                return Node(X, self.q, self.p, e, left, right, node_type = 'exNode' )
            
            # Split Point를 min(values of q), max(values of q) 중 랜덤하게 선택
            self.p = random.uniform(mini, maxi)
            
            # q가 p보다 작은 데이터들은 left node로, q가 p보다 같거나 큰 데이터들은 right node로 지정
            # 위와 같은 과정들을 재귀적으로 분기 진행
            w = np.where(X[:][self.q] < self.p, True, False)
            return Node(X, self.q, self.p, e,\
                        left=self.make_tree(X[w], e+1, l),\
                        right=self.make_tree(X[~w], e+1, l),\
                        node_type = 'inNode')

    def get_node(self, path):
        node = self.root
        for p in path:
            if p == 'L' : node = node.left
            if p == 'R' : node = node.right
        return node

### 🔎 iForest
> 그럼 이제 위에서 구축된 Isolation Tree를 가지고 본격적으로 Forest를 구축해보겠습니다! Isolation Forest란 말그대로 여러 Isolation Tree로 구성된 숲을 뜻합니다. 위에서 확인했다싶이 뭐든지 랜덤하게 선택되고, 랜덤하게 분기하니까 하나의 Tree로 얻어진 Anomaly Score를 사용하면 성능이 좋지 않겠죠..?! 그러므로 여러 Tree를 통해 얻은 Anomaly Score를 최종적으로 평균내서 사용하게 됩니다.


<img src="images/if_iforest.png" width="450">

In [7]:
def c_factor(n) :
    return 2.0*(np.log(n-1)+0.5772156649) - (2.0*(n-1.)/(n*1.0))

In [8]:
class iForest(object):
    def __init__(self, X, ntrees, sample_size, limit=None):
        self.ntrees = ntrees  # 몇 개의 Tree로 숲을 구성할지 결정
        self.X = X
        self.nobjs = len(X)
        self.sample = sample_size  # 샘플링 사이즈 (몇 개의 데이터를 이용해 각 tree를 학습할지)
        self.Trees = []
        self.limit = limit  # height limit (iTree를 구축할 때 종료 조건에 활용될 변수)
        if limit is None:
            self.limit = int(np.ceil(np.log2(self.sample)))
        self.c = c_factor(self.sample)
        # pseudo code #3 ~ #6
        # 사전 설정한 Tree 개수에 따라 각 Tree별 랜덤하게 데이터 샘플링 후, iTree 구축
        for i in tqdm(range(self.ntrees), desc="Constructing Tree..."):
            ix = random.sample(range(self.nobjs), self.sample)
            X_p = X.loc[ix]
            self.Trees.append(iTree(X_p, 0, self.limit))

    def compute_paths(self, X_in = None):
        if X_in is None:
            X_in = self.X
        S = np.zeros(len(X_in))
        for i in  tqdm(range(len(X_in)), desc="Computing path..."):
            h_temp = 0
            for j in range(self.ntrees):
                h_temp += PathFactor(X_in.loc[i], self.Trees[j]).path*1.0
            """
            Anomaly Score 계산식
            Eh = 전체 Tree에 대해 특정 Leaf Node(데이터)까지의 평균 길이 
            높을수록 데이터를 고립시키는데 많은 분기가 필요 → 정상데이터일 확률 ⬆
            낮을수록 데이터를 고립시키는데 적은 분기가 필요 → 이상치일 확률 ⬆
            """
            Eh = h_temp / self.ntrees
            S[i] = 2.0**(-Eh/self.c)
        return S

### 🔎 Path Length
> 이상치 여부를 판단하기 위해선 각 Tree 별 데이터가 고립되기까지의 평균 길이 (즉, 루트 노드로부터 데이터 노드까지의 거리)를 계산해야 합니다.    

<img src="images/if_pathlength.png" width="450">


> 💡 특히 처음 강의 자료에서 pseudo code만 보았을 땐 #1~#2이 무엇을 위한 건지 이해가 안됐는데 **height limit에 의해 분기가 중단되어 고려되지 못한 실질적인 depth를 반영하기 위해** 각 데이터 별로 c값을 더해주는 거라고 합니다!  

> 만약 leaf node에 포함된 데이터가 1개라면 완벽히 고립된 것이므로 c=1로 두고, 1개보다 크다면 height limit으로 인해 더 이상 분기하지 못한 것이니 아래와 같은 식으로 정의합니다. 이렇게 구한 c값을 각 데이터 별 leaf node의 depth에 더해주면 각 데이터 별 평균 길이를 각 Tree별로 보정하는 효과를 낼 수 있습니다.

<img src="images/if_c.png" width="300">

In [9]:
class PathFactor(object):
    def __init__(self, x, itree):
        self.path_list=[]        
        self.x = x
        self.e = 0
        self.path = self.find_path(itree.root)

    def find_path(self,T):
        if T.ntype == 'exNode':
            # Case 1 : leaf node에 데이터가 한개만 존재하는 완전히 고립된 상태
            # Current Tree Path를 그대로 반화
            if T.size == 1: return self.e
            # Case 2 : leaf node에 데이터가 두개 이상 존재하는 완전히 고립되지 못한 상태
            # Current Tree Path + c_factor 반환
            else:
                self.e = self.e + c_factor(T.size)
                return self.e
        else:
            a = T.q
            self.e += 1
            if self.x[a] < T.p:
                self.path_list.append('L')
                return self.find_path(T.left)
            else:
                self.path_list.append('R')
                return self.find_path(T.right)

### 🔎 결과 확인

In [10]:
def visualize(anomaly_score):
    # Anomaly Distribution을 시각화
    f, axes = plt.subplots(1, 1, figsize=(7, 7), sharex=True)
    sb.distplot(anomaly_score, kde=True, color="b", ax=axes, axlabel='anomaly score')
        
def show_result(anomaly_score, threshold=[0.5, 0.6, 0.7, 0.8, 0.9]):
    f1_scores = {}
    for t in threshold:
        result = np.where(anomaly_score >= t, 1, 0)
        f1_scores[t] = round(f1_score(val_dataset_label, result, average='macro'), 3)
    for t in f1_scores.keys():
        print(f'Threshold: {t} => Macro F1-Score: {f1_scores[t]}')
    print(f'⭐️ Best F1-Score : {sorted(f1_scores.items(), reverse=True, key=lambda item: item[1])[0][1]}')

- **tree 개수에 따른 성능 변화**

In [11]:
exp_ntrees = [10, 50, 100]
exp_ntrees_results = {}
exp_ntrees_forests = []
for ntree in exp_ntrees:
    print("="*100)
    print(f'Number of 🌳 : {ntree}')
    F = iForest(train_dataset, ntrees=ntree, sample_size=256)
    S_val = F.compute_paths(X_in=val_dataset_attributes)
    exp_ntrees_results[ntree] = S_val
    exp_ntrees_forests.append(F)
    show_result(S_val)

Number of 🌳 : 10


Constructing Tree...:   0%|          | 0/10 [00:00<?, ?it/s]

Computing path...:   0%|          | 0/28462 [00:00<?, ?it/s]

Threshold: 0.5 => Macro F1-Score: 0.511
Threshold: 0.6 => Macro F1-Score: 0.566
Threshold: 0.7 => Macro F1-Score: 0.648
Threshold: 0.8 => Macro F1-Score: 0.5
Threshold: 0.9 => Macro F1-Score: 0.5
⭐️ Best F1-Score : 0.648
Number of 🌳 : 50


Constructing Tree...:   0%|          | 0/50 [00:00<?, ?it/s]

Computing path...:   0%|          | 0/28462 [00:00<?, ?it/s]

Threshold: 0.5 => Macro F1-Score: 0.508
Threshold: 0.6 => Macro F1-Score: 0.561
Threshold: 0.7 => Macro F1-Score: 0.545
Threshold: 0.8 => Macro F1-Score: 0.5
Threshold: 0.9 => Macro F1-Score: 0.5
⭐️ Best F1-Score : 0.561
Number of 🌳 : 100


Constructing Tree...:   0%|          | 0/100 [00:00<?, ?it/s]

Computing path...:   0%|          | 0/28462 [00:00<?, ?it/s]

Threshold: 0.5 => Macro F1-Score: 0.506
Threshold: 0.6 => Macro F1-Score: 0.558
Threshold: 0.7 => Macro F1-Score: 0.552
Threshold: 0.8 => Macro F1-Score: 0.5
Threshold: 0.9 => Macro F1-Score: 0.5
⭐️ Best F1-Score : 0.558


In [12]:
adfadsfadf

NameError: name 'adfadsfadf' is not defined

- **sample size에 따른 성능 변화**

In [15]:
exp_ss = [128, 256, 512]
exp_ss_results = {}
exp_ss_forests = []
for ss in exp_ss:
    print("="*100)
    print(f'Number of Sample Size : {ss}')
    F = iForest(train_dataset, ntrees=10, sample_size=ss)
    S_val = F.compute_paths(X_in=val_dataset_attributes)
    exp_ss_results[ss] = S_val
    exp_ss_forests.append(F)
    show_result(S_val)

Number of Sample Size : 128


Constructing Tree...:   0%|          | 0/10 [00:00<?, ?it/s]

Computing path...:   0%|          | 0/28462 [00:00<?, ?it/s]

Threshold: 0.5 => Macro F1-Score: 0.483
Threshold: 0.6 => Macro F1-Score: 0.519
Threshold: 0.7 => Macro F1-Score: 0.499
Threshold: 0.8 => Macro F1-Score: 0.5
Threshold: 0.9 => Macro F1-Score: 0.5
⭐️ Best F1-Score : 0.519
Number of Sample Size : 256


Constructing Tree...:   0%|          | 0/10 [00:00<?, ?it/s]

Computing path...:   0%|          | 0/28462 [00:00<?, ?it/s]

Threshold: 0.5 => Macro F1-Score: 0.5
Threshold: 0.6 => Macro F1-Score: 0.554
Threshold: 0.7 => Macro F1-Score: 0.614
Threshold: 0.8 => Macro F1-Score: 0.5
Threshold: 0.9 => Macro F1-Score: 0.5
⭐️ Best F1-Score : 0.614
Number of Sample Size : 512


Constructing Tree...:   0%|          | 0/10 [00:00<?, ?it/s]

Computing path...:   0%|          | 0/28462 [00:00<?, ?it/s]

Threshold: 0.5 => Macro F1-Score: 0.515
Threshold: 0.6 => Macro F1-Score: 0.599
Threshold: 0.7 => Macro F1-Score: 0.707
Threshold: 0.8 => Macro F1-Score: 0.5
Threshold: 0.9 => Macro F1-Score: 0.5
⭐️ Best F1-Score : 0.707


---

## 2️⃣ Non-random Split using Information gain
> Information gain을 통해 Decision Tree를 구축하기 위해서 필요한 Label 정보가 현재 Train Dataset에는 없으므로 한번 학습을 진행한 Isolation Forest를 통해 Pseudo Label을 구축하겠습니다.
- Reference
    - https://community.rapidminer.com/discussion/58166/decision-tree-without-a-label-is-it-possible
    - https://towardsdatascience.com/entropy-and-information-gain-in-decision-trees-c7db67a3a293

In [16]:
def get_pseudo_label(dataset):
    # 가장 성능이 좋았던 n_tree=10, sample_size=512의 Isolation Forest를 사용하겠습니다.
    S_train = exp_ss_forests[-1].compute_paths(X_in=train_dataset)
    pseudo_label = np.where(S_train >= 0.7, 1, 0)
    return pseudo_label

In [17]:
train_dataset_copy = train_dataset.copy()

In [18]:
train_dataset_copy['pseudo_label'] = get_pseudo_label(train_dataset_copy)

Computing path...:   0%|          | 0/113842 [00:00<?, ?it/s]

In [19]:
train_dataset_label = train_dataset_copy['pseudo_label']
train_dataset_attributes = train_dataset_copy.drop(columns=['pseudo_label'])

In [ ]:
def calc_entropy(column):
    # Compute the counts of each unique value in the column
    counts = np.bincount(column)
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            # use log from math and set base to 2
            entropy += prob * math.log(prob, 2)
    
    return -entropy

In [ ]:
def calc_information_gain(data, split_name, target_name='pseudo_label'):
    # Calculate the original entropy
    original_entropy = calc_entropy(data[target_name])
    
    #Find the unique values in the column
    values = data[split_name].unique()
    
    # Make two subsets of the data, based on the unique values
    left_split = data[data[split_name] == values[0]]
    right_split = data[data[split_name] == values[1]]
    
    # Loop through the splits and calculate the subset entropies
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0]) 
        to_subtract += prob * calc_entropy(subset[target_name])
    
    # Return information gain
    return original_entropy - to_subtract

In [ ]:
def highest_info_gain(columns):
    #Intialize an empty dictionary for information gains
    information_gains = {}

    #Iterate through each column name in our list
    for col in columns:
        #Find the information gain for the column
        information_gain = calc_information_gain(midwest, col, 
        #Add the information gain to our dictionary using the column name as the ekey                                         
        information_gains[col] = information_gain

    #Return the key with the highest value                                          
    return max(information_gains, key=information_gains.get)

In [ ]:
class iTree(object):
    def __init__(self, X, e, l):
        self.e = e  # 현재 tree height
        self.X = X  # 데이터
        self.size = len(X) # 데이터에 포함된 object 개수
        self.Q = np.arange(np.shape(X)[1], dtype='int') # 데이터의 attributes
        self.l = l  # 종료 조건에 사용될 tree의 height limit
        self.p = None  # split point (max(values of q), min(values of q) 중 랜덤하게 선택됨)
        self.q = None  # split attribute (Q에서 랜덤하게 선택됨)
        self.exnodes = 0  # external node
        self.root = self.make_tree(X, e, l)
        

    def make_tree(self, X, e, l, split_criteria='random'):
        self.e = e
        
        # 종료 조건
        # 1. 현재 tree height이 사전 지정한 height limit에 도달한 경우
        # 2. 
        if e >= l or len(X) <= 1:
            left = None
            right = None
            self.exnodes += 1
            return Node(X, self.q, self.p, e, left, right, node_type = 'exNode')
        else:
            # Split하는 기준이 되는 Attribute를 데이터의 Attributes 중에서 랜덤하게 선택
            self.q = random.choice(self.Q)
            mini = X[:,self.q].min()
            maxi = X[:,self.q].max()
            if mini==maxi:
                left = None
                right = None
                self.exnodes += 1
                return Node(X, self.q, self.p, e, left, right, node_type = 'exNode' )
            
            # Split Point를 min(values of q), max(values of q) 중 랜덤하게 선택
            self.p = random.uniform(mini, maxi)
            
            # q가 p보다 작은 데이터들은 left node로, q가 p보다 같거나 큰 데이터들은 right node로 지정
            # 위와 같은 과정들을 재귀적으로 분기 진행
            w = np.where(X[:, self.q] < self.p, True, False)
            return Node(X, self.q, self.p, e,\
                        left=self.make_tree(X[w], e+1, l),\
                        right=self.make_tree(X[~w], e+1, l),\
                        node_type = 'inNode')

    def get_node(self, path):
        node = self.root
        for p in path:
            if p == 'L' : node = node.left
            if p == 'R' : node = node.right
        return node

> 유튜브 영상을 제작하면서 들었던 궁금증을 해결할 수 있었던 유익한 시간이었습니다. 그럼 다음 튜토리얼에 **`Ensemble Learning`**으로 다시 찾아뵙겠습니다.

<img src="images/9_turtle.png" width="300">